In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
df = pd.read_csv("../NBA_INTL_FIBA_Clustering.csv", encoding='latin-1', index_col=0)
all_names = ['Average'] + df.index.tolist()

# Split up into separate dataframes
FIBA_df = df[[c for c in df.columns if c.startswith('FIBA')]].dropna()
NBA_df = df[[c for c in df.columns if c.startswith('NBA')]].dropna()
INTL_df = df[[c for c in df.columns if c.startswith('INTL')]].dropna()
dfs = [FIBA_df, NBA_df, INTL_df]
for d in dfs:
    print(d.shape)

(1306, 11)
(986, 11)
(3871, 11)


In [3]:
# Normalize dataframes, add row for 'Average', put cluster series in list
cluster_series = []
for i in range(len(dfs)):
    cluster_series.append(dfs[i].iloc[:, -1])
    df = dfs[i].iloc[:, :-1]
    #df = df.rename(columns={c: c[5:] for c in df.columns})
    df.columns = ['BH', 'SU', 'TR', 'IS', 'PU', 'RM', 'CT', 'OR', 'OS', 'HO']
    df = df.div(df.sum(axis=1), axis=0) # Normalize by row
    mean_vals = df.mean().rename("Average")
    dfs[i] = pd.concat([df, pd.DataFrame(mean_vals).transpose()])

for d in dfs:
    print(d.shape)

(1307, 10)
(987, 10)
(3872, 10)


In [4]:
def get_polar_plots_league(df, players, showlegend=False):
    """
    df: A dataframe with the play type distribution for each player in a particular league
    players: A list of players to look up, possibly including 'Average'
    If a player is not in the dataframe, does not plot this player
    """
    global colours, cur_colour_idx
    
    if not players: # No players passed
        return px.line_polar(r = [0 for i in range(10)], theta = df.columns)
    
    valid_players = df.index.intersection(players)
    
    gs = []
    for i in range(len(players)):
        p = players[i]
        if p in valid_players:
            r = df.loc[p].tolist()
            theta = df.columns.tolist()
            g = go.Scatterpolar(r = r + [r[0]], theta = theta + [theta[0]], mode = 'lines', name = p,
                               line = {'color': colours[i%10]}, legendgroup = i, showlegend = (i not in existing_groups))
            existing_groups.add(i)
            gs.append(g)

    return tuple(gs)

colours = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
existing_groups = set()

In [5]:
dropdown_options = [{"label":i, "value": i} for i in all_names]

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Play Type Distribution of Players"),
    html.P("Select any number of players from the dropdown below:"),
    dcc.Dropdown(id="selected_players", options=dropdown_options, value="Average", multi=True, placeholder="Search for a player or 'Average'",),
    dcc.Graph(id='play_type_dist')])

@app.callback(
    Output('play_type_dist', 'figure'),
    [Input("selected_players", "value")])
def update_figure(players):
    global existing_groups
    existing_groups = set()
    if isinstance(players, str):
        players = [players]
    fig = make_subplots(cols=3, specs=[[{"type": "polar"}, {"type": "polar"}, {"type": "polar"}]],
                       subplot_titles=("FIBA", "NBA", "INTL"))
    for i in range(3):
        for g in get_polar_plots_league(dfs[i], players, showlegend = (i==0)):
            fig.add_trace((g), row=1, col=i+1)
    
    fig.update_polars(radialaxis_range=[0, 0.85])
    fig.update_layout(legend= {'orientation': "h", 'yanchor': "top", 'y': 0})
    return fig

app.run_server(mode="inline", debug=True, port=8057)